# Load and Validate Raw Data from AWS S3
This notebook was to  focus on reading the raw data from S3, inspecting the dataset, and validating its contents but 
the platform wasnt available at the time . Therefore we use Local File sysytem

Recommendation:
If scalability is important:

Use Google Cloud Storage (GCS) or Azure Data Lake (ADLS) for a seamless cloud alternative to AWS S3. If working locally or on a small project:
Use the local file system or MinIO for simple and lightweight storage.

In [14]:
import os
os.environ["HADOOP_HOME"] = "C:\\hadoop"
os.environ["PATH"] += os.pathsep + "C:\\hadoop\\bin"


In [16]:
# Import required libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, when, isnull

# Initialize Spark session
spark = SparkSession.builder \
    .appName("CustomerChurnDataIngestion") \
    .config("spark.hadoop.fs.file.impl.disable.cache", "true") \
    .getOrCreate()


# Load raw customer data from the local file system
file_path = r"C:\Users\ADMIN\Desktop\Projects\Batch-Processing-Project-Customer-Churn-Prediction-Pipeline\datasets\Bank Customer Churn Prediction.csv"
raw_data = spark.read.csv(f"file:///{file_path.replace('\\', '/')}", header=True, inferSchema=True)

# Display the first few rows of the raw data
print("Raw Data Sample:")
raw_data.show(5)


Raw Data Sample:
+-----------+------------+-------+------+---+------+---------+---------------+-----------+-------------+----------------+-----+
|customer_id|credit_score|country|gender|age|tenure|  balance|products_number|credit_card|active_member|estimated_salary|churn|
+-----------+------------+-------+------+---+------+---------+---------------+-----------+-------------+----------------+-----+
|   15634602|         619| France|Female| 42|     2|      0.0|              1|          1|            1|       101348.88|    1|
|   15647311|         608|  Spain|Female| 41|     1| 83807.86|              1|          0|            1|       112542.58|    0|
|   15619304|         502| France|Female| 42|     8| 159660.8|              3|          1|            0|       113931.57|    1|
|   15701354|         699| France|Female| 39|     1|      0.0|              2|          0|            0|        93826.63|    0|
|   15737888|         850|  Spain|Female| 43|     2|125510.82|              1|         

In [18]:
# Check schema of the loaded data

print("Schema of Raw Data:")
raw_data.printSchema()




Schema of Raw Data:
root
 |-- customer_id: integer (nullable = true)
 |-- credit_score: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- balance: double (nullable = true)
 |-- products_number: integer (nullable = true)
 |-- credit_card: integer (nullable = true)
 |-- active_member: integer (nullable = true)
 |-- estimated_salary: double (nullable = true)
 |-- churn: integer (nullable = true)



In [20]:
# Validate the data (e.g., check for missing values)

print("Missing Data Check:")
missing_data = raw_data.select([count(when(isnull(c), c)).alias(c) for c in raw_data.columns])
missing_data.show()

# Check for duplicate rows
total_rows = raw_data.count()
unique_rows = raw_data.distinct().count()
duplicates = total_rows - unique_rows

print(f"Total Rows: {total_rows}")
print(f"Unique Rows: {unique_rows}")
print(f"Number of Duplicate Rows: {duplicates}")



Missing Data Check:
+-----------+------------+-------+------+---+------+-------+---------------+-----------+-------------+----------------+-----+
|customer_id|credit_score|country|gender|age|tenure|balance|products_number|credit_card|active_member|estimated_salary|churn|
+-----------+------------+-------+------+---+------+-------+---------------+-----------+-------------+----------------+-----+
|          0|           0|      0|     0|  0|     0|      0|              0|          0|            0|               0|    0|
+-----------+------------+-------+------+---+------+-------+---------------+-----------+-------------+----------------+-----+

Total Rows: 10000
Unique Rows: 10000
Number of Duplicate Rows: 0


In [25]:
# Save the validated data for the next step (optional, as Parquet format)
#output_path = r"C:\Users\ADMIN\Desktop\Projects\Batch-Processing-Project-Customer-Churn-Prediction-Pipeline\datasets\validated_data.parquet"
output_path = "file:///C:/Users/ADMIN/Desktop/Projects/Batch-Processing-Project-Customer-Churn-Prediction-Pipeline/datasets/validated_data.parquet"

#raw_data.write.parquet(f"file:///{output_path.replace('\\', '/')}", mode="overwrite")
raw_data.write.parquet(output_path, mode="overwrite")


# Stop Spark session
spark.stop()

Py4JJavaError: An error occurred while calling o183.parquet.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.getAllCommittedTaskPaths(Fi

In [27]:
# Write validated data to a simpler path for testing
output_test_path = "file:///C:/Users/ADMIN/Desktop/temp_validated_data.parquet"

try:
    raw_data.write.mode("overwrite").parquet(output_test_path)
    print(f"Data written successfully to {output_test_path}")
except Exception as e:
    print("Error during write operation:", e)


Error during write operation: An error occurred while calling o144.parquet.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.getAllCommittedTaskPath

In [29]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import os

# Create a SparkSession
spark = SparkSession.builder \
    .appName("Data Ingestion") \
    .config("spark.master", "local[*]") \
    .config("spark.sql.shuffle.partitions", "4") \
    .getOrCreate()

print("Spark Session Created.")

# Set up Hadoop environment for Windows (if required)
os.environ["HADOOP_HOME"] = "C:/hadoop"
os.environ["PATH"] += os.pathsep + "C:/hadoop/bin"

# Define file paths
raw_data_path = "file:///C:/Users/ADMIN/Desktop/Projects/Batch-Processing-Project-Customer-Churn-Prediction-Pipeline/datasets/Bank Customer Churn Prediction.csv"
validated_data_path = "file:///C:/Users/ADMIN/Desktop/Projects/Batch-Processing-Project-Customer-Churn-Prediction-Pipeline/datasets/validated_data.parquet"

# Read raw data
try:
    raw_data = spark.read.option("header", True).option("inferSchema", True).csv(raw_data_path)
    print("Raw data loaded successfully.")
except Exception as e:
    print("Error while loading raw data:", e)

# Show raw data schema
raw_data.printSchema()

# Display a preview of the data
raw_data.show(5, truncate=False)

# Basic data validation
print("\nValidating Data...")
validated_data = raw_data.filter(
    col("age").isNotNull() &  # Example: Validate age column is not null
    (col("Credit_Limit") > 0)         # Example: Validate numeric values in 'Credit_Limit'
)

# Show the number of rows before and after validation
print(f"Row count before validation: {raw_data.count()}")
print(f"Row count after validation: {validated_data.count()}")

# Show validated data preview
validated_data.show(5, truncate=False)

# Write validated data to Parquet
try:
    validated_data.write.mode("overwrite").parquet(validated_data_path)
    print(f"Validated data written successfully to: {validated_data_path}")
except Exception as e:
    print("Error while writing validated data:", e)

# Stop Spark session
spark.stop()
print("\nSpark Session Stopped.")


Spark Session Created.
Raw data loaded successfully.
root
 |-- customer_id: integer (nullable = true)
 |-- credit_score: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- balance: double (nullable = true)
 |-- products_number: integer (nullable = true)
 |-- credit_card: integer (nullable = true)
 |-- active_member: integer (nullable = true)
 |-- estimated_salary: double (nullable = true)
 |-- churn: integer (nullable = true)

+-----------+------------+-------+------+---+------+---------+---------------+-----------+-------------+----------------+-----+
|customer_id|credit_score|country|gender|age|tenure|balance  |products_number|credit_card|active_member|estimated_salary|churn|
+-----------+------------+-------+------+---+------+---------+---------------+-----------+-------------+----------------+-----+
|15634602   |619         |France |Female|42 |2     |0

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `Credit_Limit` cannot be resolved. Did you mean one of the following? [`credit_card`, `credit_score`, `country`, `gender`, `tenure`].;
'Filter (isnotnull(age#808) AND ('Credit_Limit > 0))
+- Relation [customer_id#804,credit_score#805,country#806,gender#807,age#808,tenure#809,balance#810,products_number#811,credit_card#812,active_member#813,estimated_salary#814,churn#815] csv


In [31]:
# 01_data_ingestion.ipynb

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Initialize Spark session
spark = SparkSession.builder \
    .appName("CustomerChurnPrediction") \
    .getOrCreate()

# Path to the dataset
data_path = "C:/Users/ADMIN/Desktop/Projects/Batch-Processing-Project-Customer-Churn-Prediction-Pipeline/datasets/Bank Customer Churn Prediction.csv"

# Load the raw dataset
print("\nLoading data...")
raw_data = spark.read.csv(data_path, header=True, inferSchema=True)

# Show the first few rows of the dataset
print("\nInitial Data Preview:")
raw_data.show(5, truncate=False)

# Basic data validation
print("\nValidating Data...")
validated_data = raw_data.filter(
    F.col("age").isNotNull() &  # Ensure 'age' is not null
    (F.col("balance") >= 0)     # Ensure 'balance' is non-negative
)

# Show the number of rows before and after validation
print(f"Row count before validation: {raw_data.count()}")
print(f"Row count after validation: {validated_data.count()}")

# Show validated data preview
validated_data.show(5, truncate=False)

# Save the validated data to a simpler path for testing
output_test_path = "file:///C:/Users/ADMIN/Desktop/temp_validated_data.parquet"


try:
    validated_data.write.mode("overwrite").parquet(output_test_path)
    print(f"Data written successfully to {output_test_path}")
except Exception as e:
    print("Error during write operation:", e)

# Stop the Spark session
spark.stop()



Loading data...

Initial Data Preview:
+-----------+------------+-------+------+---+------+---------+---------------+-----------+-------------+----------------+-----+
|customer_id|credit_score|country|gender|age|tenure|balance  |products_number|credit_card|active_member|estimated_salary|churn|
+-----------+------------+-------+------+---+------+---------+---------------+-----------+-------------+----------------+-----+
|15634602   |619         |France |Female|42 |2     |0.0      |1              |1          |1            |101348.88       |1    |
|15647311   |608         |Spain  |Female|41 |1     |83807.86 |1              |0          |1            |112542.58       |0    |
|15619304   |502         |France |Female|42 |8     |159660.8 |3              |1          |0            |113931.57       |1    |
|15701354   |699         |France |Female|39 |1     |0.0      |2              |0          |0            |93826.63        |0    |
|15737888   |850         |Spain  |Female|43 |2     |125510.82|1 